<a href="https://colab.research.google.com/github/AnamKwon/Chatbot_data/blob/master/Text_abs_with_KoGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/SKT-AI/KoGPT2.git
%cd KoGPT2
!pip install gluonnlp mxnet-cu101 sentencepiece torch transformers

Cloning into 'KoGPT2'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 115 (delta 59), reused 63 (delta 29), pack-reused 0
Receiving objects: 100% (115/115), 552.62 KiB | 814.00 KiB/s, done.
Resolving deltas: 100% (59/59), done.
/content/KoGPT2
     |████████████████████████████████| 348kB 5.8MB/s 
     |███████████████████████████████▌| 834.1MB 1.3MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x38fda000 @  0x7fcadcce9615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
     |████████████████████████████████| 846.0MB 21kB/s 
     |████████████████████████████████| 1.1MB 54.4MB/s 
     |████████████████████████████████| 1.4MB 48.6MB/s 
     |█████████████

In [ ]:
%cd KoGPT2
import torch
import json
from gluonnlp.data import SentencepieceTokenizer
from kogpt2.utils import *
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from tqdm import tqdm, tqdm_notebook
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import transformers

from google.colab import drive
drive.mount('/content/drive/')

tok_path = get_tokenizer()
model, vocab = get_pytorch_kogpt2_model()
tok = SentencepieceTokenizer(tok_path,  num_best=0, alpha=0)
model.config.bos_token_id, model.config.eos_token_id = vocab[vocab.bos_token], vocab[vocab.eos_token]
device = torch.device('cuda:0')
model.to(device)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
using cached model
using cached model
using cached model


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

## KoGPT2 pre trained 모델 불러오기


In [ ]:
model.train()
with open('/content/drive/MyDrive/dacon_ExtSum/train.jsonl') as f :
  train_data = [json.loads(i.strip()) for i in f.readlines()]
df = []
max_len = 0
unk_cnt = 0
for d in train_data :
  data = vocab[tok(' '.join(d['article_original']))]
  summary = vocab[tok(d['abstractive'])]
  data = [0]+data+[2]+summary+[1]
  max_len = max(max_len,len(data))
  unk_cnt += data.count(5)
  if len(data) <= 1024 :
    data = data + [3] * (1024 - len(data))
    df.append(torch.tensor(data))
  else :
    df.append(torch.tensor(data[:1024]),torch.tensor(data[-1024:]))
    max_len = max(max_len,len(data))

## 학습 데이터셋 생성
# 학습 방법
[전체 문장][SEP][요약문]
형식으로 데이터 변환


In [ ]:
load_torch = torch.load('/content/drive/MyDrive/dacon_ExtSum/checkpoint_abs_1206_1-3.tar',map_location=torch.device('cpu'))
model.load_state_dict(load_torch['model_state_dict'])

<All keys matched successfully>

In [ ]:
train_set = torch.utils.data.DataLoader(df,batch_size=3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
optimizer.load_state_dict(load_torch['optimizer_state_dict'])

In [ ]:
for e in range(30,100):
  loss_val = []
  for batch_id, token_ids in enumerate(tqdm_notebook(train_set)):
    optimizer.zero_grad()
    token_ids = token_ids.to(device)
    loss = model(token_ids, labels=token_ids)[0]
    
    loss_val.append(loss.item())
    loss.backward()
    optimizer.step()
    if batch_id % 200 == 0:
        print("epoch {} batch id {} loss {}".format(e+1, batch_id+1,  loss.item()))
    if batch_id % 5000 == 0 and batch_id != 0 :
        torch.save({
                    'epoch': e,
                    'train_no': batch_id,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss':loss
                  }, '/content/drive/MyDrive/dacon_ExtSum/checkpoint_abs_1208_1-3.tar')
  print("epoch {} loss {}".format(e+1, sum(loss_val)/len(loss_val)))
  torch.save({
          'epoch': e,
          'train_no': batch_id,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss':loss
        }, '/content/drive/MyDrive/dacon_ExtSum/checkpoint_abs_1208_1-3.tar')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


epoch 31 batch id 1 loss 0.7565739154815674
epoch 31 batch id 201 loss 0.8203890919685364
epoch 31 batch id 401 loss 0.7725662589073181
epoch 31 batch id 601 loss 0.6753486394882202
epoch 31 batch id 801 loss 0.8051952123641968
epoch 31 batch id 1001 loss 0.7394315004348755
epoch 31 batch id 1201 loss 0.6045762896537781
epoch 31 batch id 1401 loss 0.6928525567054749
epoch 31 batch id 1601 loss 0.7170279026031494
epoch 31 batch id 1801 loss 0.4448026120662689
epoch 31 batch id 2001 loss 0.4449518620967865
epoch 31 batch id 2201 loss 0.5064629316329956
epoch 31 batch id 2401 loss 0.6684534549713135
epoch 31 batch id 2601 loss 0.7128517031669617
epoch 31 batch id 2801 loss 0.6042379140853882
epoch 31 batch id 3001 loss 0.6183571815490723
epoch 31 batch id 3201 loss 0.727628231048584
epoch 31 batch id 3401 loss 1.0610600709915161
epoch 31 batch id 3601 loss 0.5643719434738159
epoch 31 batch id 3801 loss 0.5771471858024597
epoch 31 batch id 4001 loss 0.8815656900405884
epoch 31 batch id 420

epoch 32 batch id 1 loss 0.7312553524971008
epoch 32 batch id 201 loss 0.8067291378974915
epoch 32 batch id 401 loss 0.7616848349571228
epoch 32 batch id 601 loss 0.6575570702552795
epoch 32 batch id 801 loss 0.7694298028945923
epoch 32 batch id 1001 loss 0.7206419706344604
epoch 32 batch id 1201 loss 0.5896249413490295
epoch 32 batch id 1401 loss 0.6595073938369751
epoch 32 batch id 1601 loss 0.6833605766296387
epoch 32 batch id 1801 loss 0.42664429545402527
epoch 32 batch id 2001 loss 0.43785133957862854
epoch 32 batch id 2201 loss 0.49075770378112793
epoch 32 batch id 2401 loss 0.6442071795463562
epoch 32 batch id 2601 loss 0.6728910803794861
epoch 32 batch id 2801 loss 0.5938675403594971
epoch 32 batch id 3001 loss 0.5813493132591248
epoch 32 batch id 3201 loss 0.7016546130180359
epoch 32 batch id 3401 loss 1.0180706977844238
epoch 32 batch id 3601 loss 0.5600523352622986
epoch 32 batch id 3801 loss 0.5747607946395874
epoch 32 batch id 4001 loss 0.847743570804596
epoch 32 batch id 

epoch 33 batch id 1 loss 0.6899174451828003
epoch 33 batch id 201 loss 0.7768508791923523
epoch 33 batch id 401 loss 0.7513276934623718
epoch 33 batch id 601 loss 0.6563324332237244
epoch 33 batch id 801 loss 0.7755836248397827
epoch 33 batch id 1001 loss 0.7132051587104797
epoch 33 batch id 1201 loss 0.5935104489326477
epoch 33 batch id 1401 loss 0.6716634631156921
epoch 33 batch id 1601 loss 0.6862664222717285
epoch 33 batch id 1801 loss 0.4317533075809479
epoch 33 batch id 2001 loss 0.43426674604415894
epoch 33 batch id 2201 loss 0.49814388155937195
epoch 33 batch id 2401 loss 0.6555768251419067
epoch 33 batch id 2601 loss 0.6844170093536377
epoch 33 batch id 2801 loss 0.5975152254104614


## GPT2 fine turning

In [ ]:
with open('/content/drive/MyDrive/dacon_ExtSum/abstractive_test_v2.jsonl') as f :
  test_data = [json.loads(i.strip()) for i in f.readlines()]
test = []
max_len = 0
for d in test_data :
  data = vocab[tok(' '.join(d['article_original']))]
  label = d['id']
  data = [0]+data+[2]
  max_len = max(max_len,len(data))
  test.append([torch.tensor(data),label])

## 테스트 셋 생성
[전체문장][SEP] 형식으로 변환

SEP 토큰 뒤로 요약문을 생성

In [ ]:
datas = {}

In [ ]:
cnt = 1
for (data, label) in test :
  summary = model.generate(data.unsqueeze(0).to(device),max_length=1025 ,bos_token_id=0,eos_token_id=1,decoder_start_token_id=2)
  summary = summary[0,len(data):].tolist()
  sent = ''
  for i in summary :
    if i == 1 :
      break
    s = vocab.idx_to_token[i]
    if s[0] == '_' :
      s = s.replace('_', ' ')
    sent += s
  datas[label] = sent
  print(cnt)
  print(len(sent), sent)
  cnt+=1

## 내장된 generate 함수를 이용하여 요약문 생성
max_length==출력될 문장의 전체길이 

bos_token_id==문장 시작

eos_token_id==문장 끝(해당 토큰을 만날시 생성자 종료)

decoder_start_token_id==요약문 생성 위치

In [ ]:
import pandas as pd
df2 = pd.DataFrame(data=datas.items(),columns=['id','summary'])
df2.summary = df2.summary.apply(lambda x : x.replace('▁',' '))

In [ ]:
df2 = pd.DataFrame(data=datas.items(),columns=['id','summary'])
df2.summary = df2.summary.apply(lambda x : x.replace('▁',' '))

In [ ]:
df2.to_csv('/content/drive/MyDrive/dacon_ExtSum/abstractive_20201206_epoch_15.csv',index=False)

In [ ]:
df2.to_csv('/content/drive/MyDrive/dacon_ExtSum/abstractive_20201208_epoch_30_3.csv',index=False)

In [ ]:
with open('/content/gene.txt') as f :
  cnt = 0
  for i in f.readlines() :
    i = i.split(' ')
    if len(i) == 1 :
      pass
    else :
      value = i[1].strip()
      datas[test[cnt][1]] = value
      cnt += 1

In [ ]:
datas

{'250366733': '▁한국농어촌공사▁당진지사의▁비R&D▁사업에▁반발하여▁지난달▁30일에▁한국농어촌공사▁당진지사에▁공공비축▁벼▁320t을▁쌓는▁시위를▁벌였다.',
 '250371241': '▁24시간▁영업하는▁신벌떼해장국과▁해장국,▁칼칼한마당▁등▁봄철을▁맞아▁손님들에게▁감사한▁마음을▁전한다.',
 '250371242': '▁지난▁4일▁기준▁송악읍의▁휘발유와▁경유▁가격은▁1년▁전보다▁78원,▁경유는▁103원,▁액화석유가스인석유가스(LPG)▁가격이▁1년▁전보다▁각각▁71원,▁127원씩▁상승한▁것으로▁나타났다.',
 '252106905': '▁어기구▁국회의원은▁지난▁17일▁열린▁정책간담회에서▁LNG▁장기수급▁안정화를▁위해▁LNG▁제5기지의▁건설이▁차질▁없이▁진행되어줄▁것을▁강조했다.',
 '252639810': '▁당진시가▁출범한▁민선6기가▁김홍장▁당진시장의▁정책▁수립▁과정에서▁각종▁위원들의▁의견수렴이▁미흡하다는▁비판이▁계속▁제기되고▁있다.',
 '252639842': '▁마을법인▁운영에▁관련해▁내홍을▁겪는▁석문면교로3리▁출신▁이장▁P씨에▁대해▁검찰은▁업무상배임혐의로▁기소중지▁처분을▁내렸다.',
 '252639891': '▁지난▁25일▁열린▁방송통신대학▁당진학과▁이·취임식에서는▁한덕선▁직전▁수석부회장이▁이임하고▁신영호▁회장이▁취임하였다.',
 '252639893': '▁생활음악협회▁당진지부는▁정기총회와▁함께▁정기총회를▁열고▁새로▁취임한▁김형태▁지부장의▁인준을▁받았다.',
 '254192288': "▁당진교육지원청에서▁지난▁6일▁예비▁중학생▁학부모▁교육이▁진행됐으며▁이한복▁원당초▁교감이▁강의했으며▁'중학교▁생활▁적응▁방법',▁'자유▁학기제▁안내'▁등의▁내용으로▁진행되었다.",
 '254232982': '▁국립농산물품질관리원은▁지난▁3년차를▁맞이해▁당진▁소장을▁당진▁소장으로▁선임하고▁고품질·안전농식품을▁제공하고▁농업인의▁소득안정을▁지원한다.',
 '254232993': '▁지난▁9일▁열린▁당진시배구협회장▁이취임식에서▁취임한▁김송원▁회장은▁전임▁회장들의▁업적을▁이어

'250366733': '▁한국농어촌공사▁당진지사의▁비R&D▁사업에▁반발하여▁지난달▁30일에▁한국농어촌공사▁당진지사에▁공공비축▁벼▁320t을▁쌓는▁시위를▁벌였다.',

 '250371241': '▁24시간▁영업하는▁신벌떼해장국과▁해장국,▁칼칼한마당▁등▁봄철을▁맞아▁손님들에게▁감사한▁마음을▁전한다.',

 '250371242': '▁지난▁4일▁기준▁송악읍의▁휘발유와▁경유▁가격은▁1년▁전보다▁78원,▁경유는▁103원,▁액화석유가스인석유가스(LPG)▁가격이▁1년▁전보다▁각각▁71원,▁127원씩▁상승한▁것으로▁나타났다.',

 '252106905': '▁어기구▁국회의원은▁지난▁17일▁열린▁정책간담회에서▁LNG▁장기수급▁안정화를▁위해▁LNG▁제5기지의▁건설이▁차질▁없이▁진행되어줄▁것을▁강조했다.',

 '252639810': '▁당진시가▁출범한▁민선6기가▁김홍장▁당진시장의▁정책▁수립▁과정에서▁각종▁위원들의▁의견수렴이▁미흡하다는▁비판이▁계속▁제기되고▁있다.',

 '252639842': '▁마을법인▁운영에▁관련해▁내홍을▁겪는▁석문면교로3리▁출신▁이장▁P씨에▁대해▁검찰은▁업무상배임혐의로▁기소중지▁처분을▁내렸다.',